In [1]:
import pandas as pd
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
import re
from  itertools import chain
from collections import Counter

# Load PNU table and table with eligible itens

In [2]:
df_pnu = pd.read_csv("DSMZ_bactnames.csv", sep="\t")
df_pnu["complete_name"] = df_pnu.apply(lambda x:'%s %s' % (x['GENUS'],x['SPECIES']),axis=1)

In [3]:
#function for obtaining the year of publication
def get_year(author):
    year_list = re.findall(r'\d\d\d\d', author)
    first_year = int(year_list[0])
    return(first_year)

df_pnu["YEAR"] = df_pnu["AUTHORS"].apply(get_year)


In [4]:
df_eligible = pd.read_csv("table_1_eligible.csv", index_col=[0]) #it reads table derived from medline file

# Validity of the extracted species names

In [5]:
text_mining_species = pd.read_csv("freq_species.csv", header=None)
text_mining_species.columns = ["species", "freq"]
text_mining_species

,species,freq
0,Mycobacterium sherrisii sp. nov,2
1,Borrelia carolinensis sp. nov,2
2,Cronobacter dublinensis sp. nov,2
3,Chitinophaga terrae sp. nov,2
4,Cronobacter turicensis sp. nov,2
5,Novosphingobium aquaticum sp. nov,2
6,Nocardioides soli sp. nov,2
7,Flavobacterium aquaticum sp. nov,2
8,Glaciimonas alpina sp. nov,2
9,Lysobacter humi sp. nov,2


In [6]:
def clean_name(name):
    list_name = name.split(" ")[0:2]
    clean_name = " ".join(list_name)
    return(clean_name)

In [7]:
text_mining_species["species_name"] = text_mining_species["species"].apply(clean_name)
text_mining_species

,species,freq,species_name
0,Mycobacterium sherrisii sp. nov,2,Mycobacterium sherrisii
1,Borrelia carolinensis sp. nov,2,Borrelia carolinensis
2,Cronobacter dublinensis sp. nov,2,Cronobacter dublinensis
3,Chitinophaga terrae sp. nov,2,Chitinophaga terrae
4,Cronobacter turicensis sp. nov,2,Cronobacter turicensis
5,Novosphingobium aquaticum sp. nov,2,Novosphingobium aquaticum
6,Nocardioides soli sp. nov,2,Nocardioides soli
7,Flavobacterium aquaticum sp. nov,2,Flavobacterium aquaticum
8,Glaciimonas alpina sp. nov,2,Glaciimonas alpina
9,Lysobacter humi sp. nov,2,Lysobacter humi


In [8]:
#Valid names available in DSMZ PNU table
valid_names = df_pnu[(pd.notna(df_pnu["SPECIES"])) & (pd.isna(df_pnu["SUBSPECIES"]))]
valid_names

,GENUS,SPECIES,SUBSPECIES,REFERENCE,STATUS,AUTHORS,REMARKS,RISK_GRP,TYPE_STRAINS,RECORD_NO,RECORD_LNK,complete_name,YEAR
0,Abditibacterium,utsteinense,NaN,68:3379,sp. nov. (VL),Tahon et al. 2018,NaN,NaN,"DSM 105287, LMG 29911, R-68213",797965,NaN,Abditibacterium utsteinense,2018
2,Abiotrophia,adiacens,NaN,45:798*,"comb. nov. (VP), homotypic syn.",(Bouvet et al. 1989) Kawamura et al. 1995,NaN,2,"ATCC 49175, CIP 103243, DSM 9848, GaD, CCUG 27...",772466,776611.0,Abiotrophia adiacens,1989
3,Abiotrophia,balaenopterae,NaN,49:503*,sp. nov. (VP),Lawson et al. 1999,NaN,NaN,"CCUG 37380, M1975/96/1, DSM 15827, ATCC 700813...",772467,776612.0,Abiotrophia balaenopterae,1999
4,Abiotrophia,defectiva,NaN,45:798*,comb. nov. (VP),(Bouvet et al. 1989) Kawamura et al. 1995,NaN,2,"ATCC 49176, CIP 103242, DSM 9849, SC10, CCUG 2...",772468,NaN,Abiotrophia defectiva,1989
5,Abiotrophia,elegans,NaN,49:1,sp. nov. (VL),Roggenkamp et al. 1999,NaN,2,"B1333, DSM 11693, ATCC 700633, CCUG 38949, CIP...",772469,776613.0,Abiotrophia elegans,1999
7,Abyssibacter,profundi,NaN,68:3424*,sp. nov. (VP),Zhou et al. 2018,NaN,NaN,"KCTC 52933, MCCC 1K03450, OUC007",797925,NaN,Abyssibacter profundi,2018
9,Abyssicoccus,albus,NaN,66:4299,sp. nov. (VL),Jiang et al. 2016,NaN,NaN,"YIM M12140, DSM 29158, CCTCC AB 2014213",794216,NaN,Abyssicoccus albus,2016
11,Abyssivirga,alkaniphila,NaN,66:1724*,sp. nov. (VP),Schouw et al. 2016,NaN,NaN,"L81, DSM 29592, JCM 30920",794021,NaN,Abyssivirga alkaniphila,2016
13,Acanthopleuribacter,pedis,NaN,58:2597*,sp. nov. (VP),Fukunaga et al. 2008,NaN,1,"FYK2218, KCTC 12899, NBRC 101209, DSM 28897",787601,NaN,Acanthopleuribacter pedis,2008
15,Acaricomes,phytoseiuli,NaN,56:465*,sp. nov. (VP),Pukall et al. 2006 emend. Nouioui et al. 2018,NaN,"1, n2","CCUG 49701, CSC, DSM 14247, JCM 14300",772478,NaN,Acaricomes phytoseiuli,2006


In [9]:
#Intersection between found species names (text mining) and valid names (DSMZ PNU)
pd.Series(list(set(text_mining_species["species_name"]).intersection(set(valid_names["complete_name"]))))

0                      Weissella kimchii
1               Bacillus stratosphericus
2              Azospirillum fermentarium
3          Brevibacterium ammoniilyticum
4               Fulvibacter tottoriensis
5           Ornithinibacillus salinisoli
6             Nonhongiella spirulinensis
7                   Paracoccus aestuarii
8                   Oceanisphaera marina
9             Pediococcus ethanolidurans
10                Roseomonas frigidaquae
11                   Prevotella maculosa
12              Microbulbifer variabilis
13                  Mesorhizobium alhagi
14                Lysinibacillus varians
15                    Phaeobacter leonis
16       Acetobacteroides hydrogenigenes
17           Flavobacterium macrobrachii
18            Leptotrichia hongkongensis
19                Halovulum dunhuangense
20          Phyllobacterium endophyticum
21             Dactylosporangium sucinum
22         Robiginitomaculum antarcticum
23                  Racemicystis persica
24         Sphin

In [34]:
10079 / 10799 #93.33% valid

0.933327159922215

# Valid names in Antonie van Leewenhoek

In [11]:
antonie = df_eligible[(df_eligible["Journal"] == "Antonie van Leeuwenhoek")].copy()

antonie["species"] = antonie["Title"].str.findall('[A-Z]\w+ [a-z]\w+ (?:sp\. nov|sp\.nov)|[A-Z]\w+ (?:sp\. nov|sp\.nov)|[A-Z]\w+ [a-z]\w+ (?:gen\. nov|gen\.nov)|[A-Z]. \w+ (?:sp\. nov|sp\.nov)') #tenta primeiro duas palavras antes do padrão, depois uma
antonie_species = pd.Series(list(chain.from_iterable(antonie["species"])))
antonie_freq_species = pd.DataFrame(antonie_species.value_counts()).reset_index()
antonie_freq_species.columns = ["species", "freq"]
antonie_freq_species["species_name"] = antonie_freq_species["species"].apply(clean_name)
antonie_freq_species

,species,freq,species_name
0,Paenibacillus beijingensis sp. nov,2,Paenibacillus beijingensis
1,Lactobacillus bombicola sp. nov,1,Lactobacillus bombicola
2,Bacillus abyssalis sp. nov,1,Bacillus abyssalis
3,Nocardioides islandiensis sp. nov,1,Nocardioides islandiensis
4,Okibacterium endophyticum sp. nov,1,Okibacterium endophyticum
5,Altererythrobacter oceanensis sp. nov,1,Altererythrobacter oceanensis
6,Roseomonas sediminicola sp. nov,1,Roseomonas sediminicola
7,Pseudonocardia serianimatus sp. nov,1,Pseudonocardia serianimatus
8,Bacillus deserti sp. nov,1,Bacillus deserti
9,Vibrio zhuhaiensis sp. nov,1,Vibrio zhuhaiensis


In [12]:
#Valid names in Antonie van Leewenhoek
pd.Series(list(set(antonie_freq_species["species_name"]).intersection(set(valid_names["complete_name"]))))

0                    Mesorhizobium soli
1                 Salinimonas lutimaris
2                    Bacillus sediminis
3               Mycobacterium neglectum
4                 Streptomyces formicae
5                 Bacillus lindianensis
6            Pseudonocardia yuanmoensis
7            Nonhongiella spirulinensis
8                  Roseovarius gaetbuli
9             Geodermatophilus siccatus
10         Aquimixticola soesokkakensis
11               Halobacillus salicampi
12            Nocardioides rubroscoriae
13                Erysipelothrix larvae
14          Lysinibacillus endophyticus
15                 Streptomyces mayteni
16                      Weissella bombi
17              Kushneria pakistanensis
18             Loktanella aestuariicola
19     Aliidiomarina haloalkalitolerans
20                Roseomonas aeriglobus
21                    Hymenobacter agri
22                    Lutibacter oceani
23         Methylobacterium pseudosasae
24                  Streptomyces aridus


In [13]:
#Valid names in Antonie
527 / 709 #74.33%

0.7433004231311706

# Valid names in Archives of Microbiology


In [14]:
archives = df_eligible[(df_eligible["Journal"] == "Archives of microbiology")].copy()

archives["species"] = archives["Title"].str.findall('[A-Z]\w+ [a-z]\w+ (?:sp\. nov|sp\.nov)|[A-Z]\w+ (?:sp\. nov|sp\.nov)|[A-Z]\w+ [a-z]\w+ (?:gen\. nov|gen\.nov)|[A-Z]. \w+ (?:sp\. nov|sp\.nov)') #tenta primeiro duas palavras antes do padrão, depois uma
archives_species = pd.Series(list(chain.from_iterable(archives["species"])))
archives_freq_species = pd.DataFrame(archives_species.value_counts()).reset_index()
archives_freq_species.columns = ["species", "freq"]
archives_freq_species["species_name"] = archives_freq_species["species"].apply(clean_name)
archives_freq_species

,species,freq,species_name
0,Themoanaerobacterium calidifontis sp. nov,1,Themoanaerobacterium calidifontis
1,Bacillus mesophilum sp. nov,1,Bacillus mesophilum
2,Sphingomonas flavus sp. nov,1,Sphingomonas flavus
3,Ochrobactrum endophyticum sp. nov,1,Ochrobactrum endophyticum
4,Bradyrhizobium brasilense sp. nov,1,Bradyrhizobium brasilense
5,Massilia humi sp. nov,1,Massilia humi
6,Chryseobacterium yeoncheonense sp. nov,1,Chryseobacterium yeoncheonense
7,Bacillus berkeleyi sp. nov,1,Bacillus berkeleyi
8,Francisella piscicida sp. nov,1,Francisella piscicida
9,Rheinheimera japonica sp. nov,1,Rheinheimera japonica


In [15]:
#Valid names in Archives of Microbiology
pd.Series(list(set(archives_freq_species["species_name"]).intersection(set(valid_names["complete_name"]))))

0         Paenibacillus wulumuqiensis
1            Bacterioplanes sanyensis
2              Streptomyces zhihengii
3              Spongiimicrobium salis
4               Francisella piscicida
5              Thalassospira lohafexi
6             Lysobacter pocheonensis
7               Sporotalea propionica
8               Thauera phenylacetica
9             Kribbella sindirgiensis
10          Sphingomonas panaciterrae
11         Cellulosimicrobium marinum
12          Porphyrobacter sanguineus
13                Pseudomonas glareae
14           Sphingorhabdus pacificus
15                  Roseospira marina
16               Bacillus catenulatus
17             Arthrobacter enclensis
18               Bacteroides sartorii
19        Pseudomonas nitritireducens
20             Wautersia numadzuensis
21        Pelotomaculum isophthalicum
22              Cupriavidus plantarum
23                Bacillus lonarensis
24    Sphingobacterium praediipecoris
25                  Azoarcus buckelii
26        Al

In [16]:
77 / 152 #50.66% valid

0.506578947368421

# Valid names in Current Microbiology

In [17]:
current = df_eligible[(df_eligible["Journal"] == "Current microbiology")].copy()

current["species"] = current["Title"].str.findall('[A-Z]\w+ [a-z]\w+ (?:sp\. nov|sp\.nov)|[A-Z]\w+ (?:sp\. nov|sp\.nov)|[A-Z]\w+ [a-z]\w+ (?:gen\. nov|gen\.nov)|[A-Z]. \w+ (?:sp\. nov|sp\.nov)') #tenta primeiro duas palavras antes do padrão, depois uma
current_species = pd.Series(list(chain.from_iterable(current["species"])))
current_freq_species = pd.DataFrame(current_species.value_counts()).reset_index()
current_freq_species.columns = ["species", "freq"]
current_freq_species["species_name"] = current_freq_species["species"].apply(clean_name)
current_freq_species

,species,freq,species_name
0,Flavobacterium shanxiense sp. nov,1,Flavobacterium shanxiense
1,Deinococcus soli sp. nov,1,Deinococcus soli
2,Bacillus thaonhiensis sp. nov,1,Bacillus thaonhiensis
3,Microbulbifer arenaceous sp. nov,1,Microbulbifer arenaceous
4,Seohaeicola nanhaiensis sp. nov,1,Seohaeicola nanhaiensis
5,Streptomyces songpinggouensis sp. nov,1,Streptomyces songpinggouensis
6,Aneurinibacillus humi sp. nov,1,Aneurinibacillus humi
7,Algivirga pacifica gen. nov,1,Algivirga pacifica
8,Desulfobulbus aggregans sp. nov,1,Desulfobulbus aggregans
9,Ottowia shaoguanensis sp. nov,1,Ottowia shaoguanensis


In [18]:
#Valid names in Current Microbiology
pd.Series(list(set(current_freq_species["species_name"]).intersection(set(valid_names["complete_name"]))))

0                     Deinococcus soli
1               Thalassiella azotovora
2                  Spongiivirga citrea
3                  Pacificamonas flava
4                  Gelidibacter flavus
5              Mesorhizobium hankyongi
6               Ezakiella massiliensis
7          Halodurantibacterium flavum
8                 Deinococcus puniceus
9     Pseudomaricurvus alkylphenolicus
10            Aeromicrobium panacisoli
11                    Spirosoma gilvum
12                    Pontibacter humi
13              Desulfovibrio legallis
14                Nocardioides dilutes
15          Pseudomonas pharmafabricae
16                 Spongiiferula fulva
17           Paradevosia shaoguanensis
18            Lutimaribacter litoralis
19                Aeromonas aquariorum
20           Pseudomonas benzenivorans
21            Klebsiella michiganensis
22                Pseudomonas prosekii
23                 Hymenobacter terrae
24          Brevundimonas mongoliensis
25             Acuticoccu

In [19]:
70 / 161 #43.48% are valid in Current Microbiology

0.43478260869565216

# Valid names in IJSEM

In [20]:
ijsem = df_eligible[(df_eligible["Journal"] == "International journal of systematic and evolutionary microbiology")].copy()

ijsem["species"] = ijsem["Title"].str.findall('[A-Z]\w+ [a-z]\w+ (?:sp\. nov|sp\.nov)|[A-Z]\w+ (?:sp\. nov|sp\.nov)|[A-Z]\w+ [a-z]\w+ (?:gen\. nov|gen\.nov)|[A-Z]. \w+ (?:sp\. nov|sp\.nov)') #tenta primeiro duas palavras antes do padrão, depois uma
ijsem_species = pd.Series(list(chain.from_iterable(ijsem["species"])))
ijsem_freq_species = pd.DataFrame(ijsem_species.value_counts()).reset_index()
ijsem_freq_species.columns = ["species", "freq"]
ijsem_freq_species["species_name"] = ijsem_freq_species["species"].apply(clean_name)
ijsem_freq_species

,species,freq,species_name
0,Glaciimonas alpina sp. nov,2,Glaciimonas alpina
1,Pseudomonas furukawaii sp. nov,2,Pseudomonas furukawaii
2,Streptococcus minor sp. nov,1,Streptococcus minor
3,Lysinibacillus halotolerans sp. nov,1,Lysinibacillus halotolerans
4,Lysobacter panaciterrae sp. nov,1,Lysobacter panaciterrae
5,Sphingobacterium changzhouense sp. nov,1,Sphingobacterium changzhouense
6,Georgenia sediminis sp. nov,1,Georgenia sediminis
7,Bacteroides faecis sp. nov,1,Bacteroides faecis
8,Catenisphaera adipataccumulans gen. nov,1,Catenisphaera adipataccumulans
9,Microbacterium hydrothermale sp. nov,1,Microbacterium hydrothermale


In [21]:
#Valid names in IJSEM
pd.Series(list(set(ijsem_freq_species["species_name"]).intersection(set(valid_names["complete_name"]))))

0                     Weissella kimchii
1              Bacillus stratosphericus
2             Azospirillum fermentarium
3         Brevibacterium ammoniilyticum
4              Fulvibacter tottoriensis
5          Ornithinibacillus salinisoli
6                  Paracoccus aestuarii
7                  Oceanisphaera marina
8            Pediococcus ethanolidurans
9                Roseomonas frigidaquae
10                  Prevotella maculosa
11                 Mesorhizobium alhagi
12             Microbulbifer variabilis
13               Lysinibacillus varians
14                   Phaeobacter leonis
15      Acetobacteroides hydrogenigenes
16          Flavobacterium macrobrachii
17               Halovulum dunhuangense
18         Phyllobacterium endophyticum
19        Robiginitomaculum antarcticum
20                 Racemicystis persica
21        Sphingobacterium yanglingense
22                 Idiomarina salinarum
23               Oceanicola antarcticus
24        Mycobacterium psychrotolerans


In [22]:
8593 / 8736 #98.36% are valid

0.9836309523809523

# Valid names in Journal_of_microbiology_(Seoul,_Korea)


In [23]:
jom = df_eligible[(df_eligible["Journal"] == "Journal of microbiology (Seoul, Korea)")].copy()

jom["species"] = jom["Title"].str.findall('[A-Z]\w+ [a-z]\w+ (?:sp\. nov|sp\.nov)|[A-Z]\w+ (?:sp\. nov|sp\.nov)|[A-Z]\w+ [a-z]\w+ (?:gen\. nov|gen\.nov)|[A-Z]. \w+ (?:sp\. nov|sp\.nov)') #tenta primeiro duas palavras antes do padrão, depois uma
jom_species = pd.Series(list(chain.from_iterable(jom["species"])))
jom_freq_species = pd.DataFrame(jom_species.value_counts()).reset_index()
jom_freq_species.columns = ["species", "freq"]
jom_freq_species["species_name"] = jom_freq_species["species"].apply(clean_name)
jom_freq_species

,species,freq,species_name
0,Flavisolibacter swuensis sp. nov,1,Flavisolibacter swuensis
1,Sphingomonas rosea sp. nov,1,Sphingomonas rosea
2,Deinococcus rubellus sp. nov,1,Deinococcus rubellus
3,Arcobacter acticola sp. nov,1,Arcobacter acticola
4,Sphingosinicella ginsenosidimutans sp. nov,1,Sphingosinicella ginsenosidimutans
5,Psychroflexus lacisalsi sp. nov,1,Psychroflexus lacisalsi
6,Spirosoma pulveris sp. nov,1,Spirosoma pulveris
7,Lysobacter pedocola sp. nov,1,Lysobacter pedocola
8,Spirosoma montaniterrae sp. nov,1,Spirosoma montaniterrae
9,Flavobacterium jejuensis sp. nov,1,Flavobacterium jejuensis


In [24]:
#Valid names in Journal of microbiology
pd.Series(list(set(jom_freq_species["species_name"]).intersection(set(valid_names["complete_name"]))))

0                Spirosoma montaniterrae
1                     Acinetobacter soli
2             Burkholderia denitrificans
3                      Chujaibacter soli
4                   Halomonas tibetensis
5                  Deinococcus sedimenti
6                 Deinococcus seoulensis
7          Diaminobutyricimonas aerilata
8                       Limnobacter humi
9            Hymenobacter jeollabukensis
10                     Bacillus spongiae
11                     Arthrobacter soli
12                Paenibacillus swuensis
13              Pseudaminobacter granuli
14              Microbacterium suwonense
15              Homoserinimonas aerilata
16                Paenibacillus pinihumi
17                  Rhodopirellula rosea
18              Flavobacterium aquaticum
19                Acinetobacter brisouii
20    Sphingosinicella ginsenosidimutans
21               Paenibacillus camelliae
22                     Larkinella roseus
23                   Ruminococcus faecis
24             W

In [25]:
92 / 172 #53.49% are valid in Journal of Microbiology

0.5348837209302325

# Valid names in SAM

In [26]:
sam = df_eligible[(df_eligible["Journal"] == "Systematic and applied microbiology")].copy()

sam["species"] = sam["Title"].str.findall('[A-Z]\w+ [a-z]\w+ (?:sp\. nov|sp\.nov)|[A-Z]\w+ (?:sp\. nov|sp\.nov)|[A-Z]\w+ [a-z]\w+ (?:gen\. nov|gen\.nov)|[A-Z]. \w+ (?:sp\. nov|sp\.nov)') #tenta primeiro duas palavras antes do padrão, depois uma
sam_species = pd.Series(list(chain.from_iterable(sam["species"])))
sam_freq_species = pd.DataFrame(sam_species.value_counts()).reset_index()
sam_freq_species.columns = ["species", "freq"]
sam_freq_species["species_name"] = sam_freq_species["species"].apply(clean_name)
sam_freq_species

,species,freq,species_name
0,Kytococcus aerolatus sp. nov,1,Kytococcus aerolatus
1,Hansschlegelia plantiphila gen. nov,1,Hansschlegelia plantiphila
2,Vibrio bivalvicida sp. nov,1,Vibrio bivalvicida
3,L. chiayiensis sp. nov,1,L. chiayiensis
4,Arcobacter ellisii sp. nov,1,Arcobacter ellisii
5,Tsukamurella spumae sp. nov,1,Tsukamurella spumae
6,Neptuniibacter marinus sp. nov,1,Neptuniibacter marinus
7,Psychrobacter proteolyticus sp. nov,1,Psychrobacter proteolyticus
8,Pseudomonas indoloxydans sp. nov,1,Pseudomonas indoloxydans
9,Enterobacter ludwigii sp. nov,1,Enterobacter ludwigii


In [27]:
#Valid names in SAM
pd.Series(list(set(sam_freq_species["species_name"]).intersection(set(valid_names["complete_name"]))))

0           Pseudorhizobium pelagicum
1      Burkholderia kirstenboschensis
2             Pseudomonas entomophila
3                   Ruminococcus luti
4          Photobacterium malacitanum
5          Chimaereicella alkaliphila
6                   Nevskia aquatilis
7         Bifidobacterium crudilactis
8               Tepidimonas thermarum
9                Lacinutrix venerupis
10          Amycolatopsis balhimycina
11             Rhodococcus phenolicus
12               Actinomadura meyerii
13        Geobacillus galactosidasius
14                 Xenorhabdus innexi
15                 Vibrio gallaecicus
16          Clostridium saccharogumia
17      Methyloligella solikamskensis
18                      Nocardia alba
19            Herminiimonas aquatilis
20             Achromobacter animicus
21            Rhodopirellula lusitana
22             Neptuniibacter marinus
23            Croceibacter atlanticus
24             Caldimonas taiwanensis
25           Photobacterium piscicola
26          

In [28]:
325 / 369 #88.08% are valid in SAM

0.8807588075880759

# Proportion of names validated between 2001 and 2018 that were contemplated in the study

In [29]:
#Names validated between 2001 and 2018
valid_names_XXI = df_pnu[(pd.notna(df_pnu["SPECIES"])) & (pd.isna(df_pnu["SUBSPECIES"])) & (df_pnu["YEAR"] > 2000)
                         & (df_pnu["YEAR"] < 2019)]
valid_names_XXI["complete_name"]

0          Abditibacterium utsteinense
7                Abyssibacter profundi
9                   Abyssicoccus albus
11             Abyssivirga alkaniphila
13           Acanthopleuribacter pedis
15              Acaricomes phytoseiuli
18       Acetanaerobacterium elongatum
20                 Acetatifactor muris
30           Acetoanaerobium pronyense
39              Acetobacter cerevisiae
40           Acetobacter cibinongensis
44                 Acetobacter fabarum
45               Acetobacter farinalis
46               Acetobacter ghanensis
48           Acetobacter indonesiensis
50                 Acetobacter lambici
53                 Acetobacter malorum
55                   Acetobacter musti
56         Acetobacter nitrogenifigens
58                    Acetobacter oeni
59             Acetobacter okinawensis
60              Acetobacter orientalis
63                 Acetobacter papayae
71                 Acetobacter persici
72                Acetobacter persicus
74            Acetobacter

In [30]:
#Intersection between found species names (text mining) and names validated in XXI century (DSMZ PNU)
pd.Series(list(set(text_mining_species["species_name"]).intersection(set(valid_names_XXI["complete_name"]))))

0                     Weissella kimchii
1              Bacillus stratosphericus
2             Azospirillum fermentarium
3         Brevibacterium ammoniilyticum
4              Fulvibacter tottoriensis
5          Ornithinibacillus salinisoli
6            Nonhongiella spirulinensis
7                  Paracoccus aestuarii
8                  Oceanisphaera marina
9            Pediococcus ethanolidurans
10               Roseomonas frigidaquae
11                  Prevotella maculosa
12             Microbulbifer variabilis
13                 Mesorhizobium alhagi
14               Lysinibacillus varians
15                   Phaeobacter leonis
16      Acetobacteroides hydrogenigenes
17          Flavobacterium macrobrachii
18           Leptotrichia hongkongensis
19               Halovulum dunhuangense
20         Phyllobacterium endophyticum
21            Dactylosporangium sucinum
22        Robiginitomaculum antarcticum
23                 Racemicystis persica
24        Sphingobacterium yanglingense


In [31]:
9962 / 12006 #82.98 % of the names validated between 2001 and 2018 were contemplated in the study

0.829751790771281